## Bayesian Networks

Bayesian networks are a powerful inference tool, in which a set of variables are represented as nodes, and the lack of an edge represents a conditional independence statement between the two variables, and an edge represents a dependence between the two variables. One of the powerful components of a Bayesian network is the ability to infer the values of certain variables, given observed values for another set of variables. These are referred to as the 'hidden' and 'observed' variables respectively, and need not be set at the time the network is created. The same network can have a different set of variables be hidden or observed between two data points. The more values which are observed, the closer the inferred values will be to the truth.

In [ ]:
!pip install pomegranate

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy

from pomegranate import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)


## The Monty Hall Gameshow

The Monty Hall problem arose from the gameshow Let's Make a Deal, where a guest had to choose which one of three doors had a prize behind it. The twist was that after the guest chose, the host, originally Monty Hall, would then open one of the doors the guest did not pick and ask if the guest wanted to switch which door they had picked. Initial inspection may lead you to believe that if there are only two doors left, there is a 50-50 chance of you picking the right one, and so there is no advantage one way or the other. However, it has been proven both through simulations and analytically that there is in fact a 66% chance of getting the prize if the guest switches their door, regardless of the door they initially went with.

We can reproduce this result using Bayesian networks with three nodes, one for the guest, one for the prize, and one for the door Monty chooses to open. The door the guest initially chooses and the door the prize is behind are completely random processes across the three doors, but the door which Monty opens is dependent on both the door the guest chooses (it cannot be the door the guest chooses), and the door the prize is behind (it cannot be the door with the prize behind it).

To create the Bayesian network in pomegranate, we first create the distributions which live in each node in the graph. For a discrete (aka categorical) bayesian network we use DiscreteDistribution objects for the root nodes and ConditionalProbabilityTable objects for the inner and leaf nodes. The columns in a ConditionalProbabilityTable correspond to the order in which the parents (the second argument) are specified, and the last column is the value the ConditionalProbabilityTable itself takes. In the case below, the first column corresponds to the value 'guest' takes, then the value 'prize' takes, and then the value that 'monty' takes. 'B', 'C', 'A' refers then to the probability that Monty reveals door 'A' given that the guest has chosen door 'B' and that the prize is actually behind door 'C', or P(Monty='A'|Guest='B', Prize='C').


In [ ]:
# The guests initial door selection is completely random
guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# The door the prize is behind is also completely random
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# Monty is dependent on both the guest and the prize. 
monty = ConditionalProbabilityTable(
        [[ 'A', 'A', 'A', 0.0 ],
         [ 'A', 'A', 'B', 0.5 ],
         [ 'A', 'A', 'C', 0.5 ],
         [ 'A', 'B', 'A', 0.0 ],
         [ 'A', 'B', 'B', 0.0 ],
         [ 'A', 'B', 'C', 1.0 ],
         [ 'A', 'C', 'A', 0.0 ],
         [ 'A', 'C', 'B', 1.0 ],
         [ 'A', 'C', 'C', 0.0 ],
         [ 'B', 'A', 'A', 0.0 ],
         [ 'B', 'A', 'B', 0.0 ],
         [ 'B', 'A', 'C', 1.0 ],
         [ 'B', 'B', 'A', 0.5 ],
         [ 'B', 'B', 'B', 0.0 ],
         [ 'B', 'B', 'C', 0.5 ],
         [ 'B', 'C', 'A', 1.0 ],
         [ 'B', 'C', 'B', 0.0 ],
         [ 'B', 'C', 'C', 0.0 ],
         [ 'C', 'A', 'A', 0.0 ],
         [ 'C', 'A', 'B', 1.0 ],
         [ 'C', 'A', 'C', 0.0 ],
         [ 'C', 'B', 'A', 1.0 ],
         [ 'C', 'B', 'B', 0.0 ],
         [ 'C', 'B', 'C', 0.0 ],
         [ 'C', 'C', 'A', 0.5 ],
         [ 'C', 'C', 'B', 0.5 ],
         [ 'C', 'C', 'C', 0.0 ]], [guest, prize])

Next, we pass these distributions into state objects along with the name for the node.

In [ ]:
# State objects hold both the distribution, and a high level name.
s1 = State(guest, name="guest")
s2 = State(prize, name="prize")
s3 = State(monty, name="monty")

# Create the Bayesian network object with a useful name
model = BayesianNetwork("Monty Hall Problem")

# Add the three states to the network 
model.add_states(s1, s2, s3)

# Add edges which represent conditional dependencies, where the second node is 
# conditionally dependent on the first node (Monty is dependent on both guest and prize)
model.add_edge(s1, s3)
model.add_edge(s2, s3)

# End of network
model.bake()


## Predicting Probabilities

We can calculate probabilities of a sample under the Bayesian network in the same way that we can calculate probabilities under other models. In this case, let's calculate the probability that you initially said door A, that Monty then opened door B, but that the actual car was behind door C.


In [ ]:
model.probability([['A', 'B', 'C']])

0.11111111111111109

The next step is to make predictions using this model. One of the strengths of Bayesian networks is their ability to infer the values of arbitrary ‘hidden variables’ given the values from ‘observed variables.’ These hidden and observed variables do not need to be specified beforehand, and the more variables which are observed the better the inference will be on the hidden variables.

In [ ]:
beliefs = model.predict_proba({ 'guest' : 'A' })
beliefs = map(str, beliefs)
print("\n".join( "{}\t{}".format( state.name, belief ) for state, belief in zip( model.states, beliefs ) ))

guest	A
prize	{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "A" : 0.3333333333333333,
            "B" : 0.3333333333333333,
            "C" : 0.3333333333333333
        }
    ],
    "frozen" : false
}
monty	{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "A" : 0.0,
            "B" : 0.49999999999999983,
            "C" : 0.49999999999999983
        }
    ],
    "frozen" : false
}


Bayesian networks are frequently used to infer/impute the value of missing variables given the observed values. In other models, typically there is either a single or fixed set of missing variables, such as latent factors, that need to be imputed, and so returning a fixed vector or matrix as the predictions makes sense.

In [ ]:
model.predict([['A', 'C', None]])

[array(['A', 'C', 'B'], dtype=object)]



---
# Lab 06
Implement the alarm network (Slide page 27) using Bayesian network, and answer following questions;



1.   What is the probability that the alarm has sounded but neither a burglary nor an earthquake has occurred, and both Mary and John call?
2.   What is the probability that the alarm has sounded by a burglary and John call?
3.   What is the probability that the alarm has sounded by a earthquake and Mary call?


In [ ]:
#Juntorn Thiantanukij 2021610213

burglary = DiscreteDistribution({'B': 0.001, 'BF': 0.999})

eathquake = DiscreteDistribution({'E': 0.002, 'EF': 0.998})

#Alarm is dependent on both the burglary and the eathquake. 
alarm = ConditionalProbabilityTable(
        [[ 'B', 'E', 'A', 0.95 ],
         [ 'B', 'E', 'AF', 0.05 ],
         [ 'B', 'EF', 'A', 0.94 ],
         [ 'B', 'EF', 'AF', 0.06 ],
         [ 'BF', 'E', 'A', 0.29 ],
         [ 'BF', 'E', 'AF', 0.71 ],
         [ 'BF', 'EF', 'A', 0.001 ],
         [ 'BF', 'EF', 'AF', 0.999 ]], [burglary, eathquake])

john = ConditionalProbabilityTable(
        [[ 'A', 'J', 0.9 ],
         [ 'A', 'JF', 0.1 ],
         [ 'AF', 'J', 0.05 ],
         [ 'AF', 'JF', 0.95 ]], [alarm])

mary = ConditionalProbabilityTable(
        [[ 'A', 'M', 0.7 ],
         [ 'A', 'MF', 0.3 ],
         [ 'AF', 'M', 0.01 ],
         [ 'AF', 'MF', 0.99 ]], [alarm])


# State objects hold both the distribution, and a high level name.
s1 = State(burglary, name="burglary")
s2 = State(eathquake, name="eathquake")
s3 = State(alarm, name="alarm")
s4 = State(john, name="john")
s5 = State(mary, name="mary")

# Create the Bayesian network object with a useful name
model = BayesianNetwork("Alarm Network Problem")

# Add the three states to the network 
model.add_states(s1, s2, s3, s4, s5)

# Add edges which represent conditional dependencies, where the second node is 
# conditionally dependent on the first node (Monty is dependent on both guest and prize)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.add_edge(s3, s4)
model.add_edge(s3, s5)

# End of network
model.bake()

In [ ]:
print("--------------------------------")
print("Juntorn Thiantanukij 2021610213")
print("--------------------------------")

print("")
print("Q1. What is the probability that the alarm has sounded but neither a bulglary nor an earthquake has occured, and both Mary and John call?")
model.probability([['BF', 'EF', 'A','J','M']])

--------------------------------
Juntorn Thiantanukij 2021610213
--------------------------------

Q1. What is the probability that the alarm has sounded but neither a bulglary nor an earthquake has occured, and both Mary and John call?


0.0006281112600000005

In [ ]:
#Redo
print("Q2. What is the probability that the alarm has sounded by a burglary and John call? ")
model.probability([['B', 'E', 'A', 'J', 'M']]) + model.probability([['B', 'EF', 'A','J', 'M']]) + model.probability([['B', 'E', 'A','J', 'MF']]) + model.probability([['B', 'EF', 'A','J', 'MF']])

Q2. What is the probability that the alarm has sounded by a burglary and John call? 


0.0008460180000000002

In [ ]:
#Redo
print("Q3. What is the probability that the alarm has sounded by a earthquake and Mary call?")
model.probability([['B', 'E', 'A', 'J', 'M']]) + model.probability([['BF', 'E', 'A','J', 'M']]) + model.probability([['B', 'E', 'A','JF', 'M']]) + model.probability([['BF', 'E', 'A','JF', 'M']])

Q3. What is the probability that the alarm has sounded by a earthquake and Mary call?


0.00040692400000000024